<a href="https://colab.research.google.com/github/alfie1104/deeplearning-with-pytorch/blob/main/pytorch_lightning/pytorch_lightning_MNIST_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 27.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [2]:
!pip install torchmetrics

# PyTorch Lightning

1. model
2. optimizer
3. data
4. training loop "the magic"
5. validation loop "the validation magic"

In [3]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [4]:
import pytorch_lightning as pl
from torchmetrics.functional import accuracy

class ResNet(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(28*28, 64)
    self.l2 = nn.Linear(64,64)
    self.l3 = nn.Linear(64,10)
    self.do = nn.Dropout(0.1)
    self.loss = nn.CrossEntropyLoss()
    self.val_step_outputs = []

  def forward(self, x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    do = self.do(h2+h1) # if h2 is not necessary, the network will set the bias of module l2 to very negative value, such that the ReLU gonna be setting to zero the value from L2.
    logits = self.l3(do)
    return logits

  # optimizer
  def configure_optimizers(self):
    optimizer = optim.SGD(self.parameters(), lr=1e-2)
    return optimizer

  # training loop
  def training_step(self, batch, batch_idx):
    x, y = batch

    # x : batch size x channel x 28 x 28
    b = x.size(0) # batch size
    x = x.view(b, -1) # in the PyTorch Lightning, we don't need CUDA stuff, even if we want to run on CUDA

    # 1) forward
    logits = self(x)

    # 2) compute the objective function
    J = self.loss(logits, y)

    acc = accuracy(logits, y, task="multiclass", num_classes=10)
    pbar = {'train_acc':acc}
    return {'loss':J, 'progress_bar':pbar} # return {'loss':J} is same as return J in PyTorch-lightning

  def validation_step(self, batch, batch_idx):
    results = self.training_step(batch, batch_idx)
    results['progress_bar']['val_acc'] = results['progress_bar']['train_acc']
    del results['progress_bar']['train_acc']
    return results

  def on_validation_epoch_end(self):
    # [results, results, results, ...] = outputs
    avg_val_loss = torch.tensor([x['loss'] for x in self.val_step_outputs]).mean()
    avg_val_acc = torch.tensor([x['progress_bar']['val_acc'] for x in self.val_step_outputs]).mean()

    self.val_step_outputs.clear() # free memory

    pbar = {'avg_val_acc':avg_val_acc}
    return {'val_loss':avg_val_loss, "progress_bar":pbar}

  def prepare_data(self):
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())

  def setup(self, stage=None):
    # Train, Val split
    dataset = datasets.MNIST('data', train=True, download=False, transform=transforms.ToTensor())
    self.train_data, self.val_data = random_split(dataset, [55000, 5000])

  def train_dataloader(self):
    train_loader = DataLoader(self.train_data, batch_size=32)
    return train_loader

  def val_dataloader(self):
    val_loader = DataLoader(self.val_data, batch_size=32)
    return val_loader


model = ResNet()

In [5]:
# trainer = pl.Trainer(max_epochs=5) # the default epochs of Trainer is 1000
trainer = pl.Trainer(max_epochs=5, devices=1, accelerator='gpu') # training on GPU
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4527902.37it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 133843.46it/s]


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [6]:
# log data (in lightning_logs/version_xx/checkpoints folder, there is the best check point model)
# ! ls lightning_logs/version_3/checkpoints
!ls lightning_logs/

version_0
